### Showing one trajectory at Rondon's head_motion_prediction

In [ ]:
import os
import sys

project_path = f"predict360user/head_motion_prediction/"
sys.path.append(project_path)
from predict360user.head_motion_prediction.David_MMSys_18.Read_Dataset import (load_sampled_dataset, plot_3d_trace)

ONE_USER = '0'
ONE_VIDEO = '10_Cows'
if not 'SAMPLE_DATASET' in locals():
  print(f"get SAMPLE_DATASET from {project_path}")
  cwd = os.getcwd()
  os.chdir(project_path) # load_sampled_datasetuses relative path
  SAMPLE_DATASET = load_sampled_dataset()
  os.chdir(cwd)

traces = SAMPLE_DATASET[ONE_USER][ONE_VIDEO][:, 1:]

# plot 3d
plot_3d_trace(traces, ONE_USER, ONE_VIDEO)


In [ ]:
import plotly.express as px
import plotly.graph_objs as go
import numpy as np
from predict360user.head_motion_prediction import utils


# plot heatmap
heatmaps = []
for i in traces:
    heatmaps.append(
        utils.from_position_to_tile(
            utils.eulerian_in_range(*utils.cartesian_to_eulerian(i[0], i[1], i[2])),
            6,
            4,
        )
    )
fig = px.imshow(
    np.sum(heatmaps, axis=0),
    labels=dict(x="longitude", y="latitude", color="requests"),
    title=f"{len(traces)}_traces (reqs={np.sum(heatmaps)})",
)
fig.update_layout(go.Layout(width=600))
fig.show()

### Dataset

In [ ]:
from predict360user import Dataset

ds = Dataset(dataset_name="all")
display(ds.df.head())
display(ds.df["actS_c"].value_counts())

In [ ]:
ds.show_entropy_histogram()

In [ ]:
ds.partition()
ds.show_entropy_histogram_per_partition()
ds.partition("low")
ds.show_entropy_histogram_per_partition()
ds.partition("medium")
ds.show_entropy_histogram_per_partition()
ds.partition("nolow")
ds.show_entropy_histogram_per_partition()
ds.partition("nohigh")
ds.show_entropy_histogram_per_partition()
ds.partition("high")
ds.show_entropy_histogram_per_partition()

### Dataset.show_traject

In [ ]:
# show random traject 
ds.show_traject(ds.df.sample(1).iloc[0])

In [ ]:
# show one traject
ds.show_traject(ds.df.loc["david","0", "10_Cows"])

### Dataset.show_traject from entropy class

In [ ]:
# show representive trajectory for each entropy class
ds.show_traject(ds.df[ds.df['actS_c'] == 'low'].nsmallest(1,'actS').iloc[0])
ds.show_traject(ds.df[ds.df['actS_c'] == 'medium'].nsmallest(1,'actS').iloc[0])
ds.show_traject(ds.df[ds.df['actS_c'] == 'high'].nlargest(1,'actS').iloc[0])

### Dataset.show_traject from poles_prc class

In [ ]:
# calculate poles_prc_c
if not 'poles_prc_c' in ds.df.columns:
  ds.calc_traces_poles_prc()
ds.show_histogram(['poles_prc'])

In [ ]:
# show representive trajectory for poler_prc class
ds.show_traject(ds.df[ds.df['poles_prc_c'] == 'low'].nsmallest(1,'actS').iloc[0])
ds.show_traject(ds.df[ds.df['poles_prc_c'] == 'medium'].nsmallest(1,'actS').iloc[0])
ds.show_traject(ds.df[ds.df['poles_prc_c'] == 'high'].nlargest(1,'actS').iloc[0])